In [1]:
from io import StringIO
import requests
import seaborn
import json
import pandas as pd
import re
from collections import defaultdict
import timeit
import matplotlib.pyplot
import numpy

%matplotlib inline

In [2]:
df_data_2 = pd.read_csv('h1b_kaggle.csv')

In [3]:
df_data_2.head(10)

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,1,CERTIFIED-WITHDRAWN,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,N,36067.0,2016.0,"ANN ARBOR, MICHIGAN",-83.743038,42.280826
1,2,CERTIFIED-WITHDRAWN,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,242674.0,2016.0,"PLANO, TEXAS",-96.698886,33.019843
2,3,CERTIFIED-WITHDRAWN,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES,CHIEF PROCESS OFFICER,Y,193066.0,2016.0,"JERSEY CITY, NEW JERSEY",-74.077642,40.728158
3,4,CERTIFIED-WITHDRAWN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES,"REGIONAL PRESIDEN, AMERICAS",Y,220314.0,2016.0,"DENVER, COLORADO",-104.990251,39.739236
4,5,WITHDRAWN,PEABODY INVESTMENTS CORP.,CHIEF EXECUTIVES,PRESIDENT MONGOLIA AND INDIA,Y,157518.4,2016.0,"ST. LOUIS, MISSOURI",-90.199404,38.627003
5,6,CERTIFIED-WITHDRAWN,BURGER KING CORPORATION,CHIEF EXECUTIVES,"EXECUTIVE V P, GLOBAL DEVELOPMENT AND PRESIDEN...",Y,225000.0,2016.0,"MIAMI, FLORIDA",-80.191790,25.761680
6,7,CERTIFIED-WITHDRAWN,BT AND MK ENERGY AND COMMODITIES,CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,91021.0,2016.0,"HOUSTON, TEXAS",-95.369803,29.760427
7,8,CERTIFIED-WITHDRAWN,"GLOBO MOBILE TECHNOLOGIES, INC.",CHIEF EXECUTIVES,CHIEF OPERATIONS OFFICER,Y,150000.0,2016.0,"SAN JOSE, CALIFORNIA",-121.886329,37.338208
8,9,CERTIFIED-WITHDRAWN,ESI COMPANIES INC.,CHIEF EXECUTIVES,PRESIDENT,Y,127546.0,2016.0,"MEMPHIS, TEXAS",NaN,NaN
9,10,WITHDRAWN,LESSARD INTERNATIONAL LLC,CHIEF EXECUTIVES,PRESIDENT,Y,154648.0,2016.0,"VIENNA, VIRGINIA",-77.265260,38.901222


In [4]:
df_data_2[['CASE_STATUS', 'SOC_NAME']]
#use this to make a chart with two indexes

,CASE_STATUS,SOC_NAME
0,CERTIFIED-WITHDRAWN,BIOCHEMISTS AND BIOPHYSICISTS
1,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES
2,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES
3,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES
4,WITHDRAWN,CHIEF EXECUTIVES
5,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES
6,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES
7,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES
8,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES
9,WITHDRAWN,CHIEF EXECUTIVES


In [5]:
df_data_2['SOC_NAME'].value_counts()

Computer Systems Analysts                                  291170
Computer Programmers                                       226574
SOFTWARE DEVELOPERS, APPLICATIONS                          221783
COMPUTER SYSTEMS ANALYSTS                                  215353
Software Developers, Applications                          192933
COMPUTER PROGRAMMERS                                       171972
COMPUTER OCCUPATIONS, ALL OTHER                            108555
Computer Occupations, All Other                             69315
Software Developers, Systems Software                       44500
SOFTWARE DEVELOPERS, SYSTEMS SOFTWARE                       39356
Management Analysts                                         38756
Financial Analysts                                          34141
Computer Software Engineers, Applications                   33325
Accountants and Auditors                                    30708
MANAGEMENT ANALYSTS                                         28117
Mechanical

In [8]:
df_data_2['SOC_NAME'].value_counts().count()

2132

In [9]:
cleandata1=df_data_2.copy()

In [10]:
cleandata1['SOC_NAME']=df_data_2['SOC_NAME'].str.lower()
cleandata1['EMPLOYER_NAME']=df_data_2['EMPLOYER_NAME'].str.lower()
cleandata1['JOB_TITLE']=df_data_2['JOB_TITLE'].str.lower()

In [13]:
cleandata1.to_csv('h1b_kaggle_lowwered.csv', sep='\t')


Here, we transform some strings to lowercase. This is because there are duplicate entries in the dataset which in both upper and lower.
This increases redundancy

In [9]:
cleandata1['SOC_NAME'].value_counts()

computer systems analysts                                             506523
software developers, applications                                     414716
computer programmers                                                  398546
computer occupations, all other                                       177870
software developers, systems software                                  83856
management analysts                                                    66873
financial analysts                                                     53540
accountants and auditors                                               52822
mechanical engineers                                                   44236
network and computer systems administrators                            40087
database administrators                                                38982
market research analysts and marketing specialists                     37737
electronics engineers, except computer                                 36574

There is still alot of redundancy we can exploit. We can generalize these strings remove specalized strings into more general form.
For example 'software engineer, senior' needs to be reduced to 'software engineer'.
This also applies to the other columns with string attributes.

In [10]:
cleandata1['SOC_NAME'].value_counts().count()

1585

We have reduced the number of names down to 1585

In [11]:
reducedf = pd.DataFrame({'SOC_NAME': cleandata1['SOC_NAME'].value_counts().index, 'Count':cleandata1['SOC_NAME'].value_counts().values})
#df['Counts'] = df.groupby(['SOC_NAME'])['Count'].transform('count') #I don't remember what I was trying to do here.
#df = df.set_index(['SOC_NAME'])
reducedf

,Count,SOC_NAME
0,506523,computer systems analysts
1,414716,"software developers, applications"
2,398546,computer programmers
3,177870,"computer occupations, all other"
4,83856,"software developers, systems software"
5,66873,management analysts
6,53540,financial analysts
7,52822,accountants and auditors
8,44236,mechanical engineers
9,40087,network and computer systems administrators


In [12]:
reducedf['Name1'] = ''

In [13]:
reducedf

,Count,SOC_NAME,Name1
0,506523,computer systems analysts,
1,414716,"software developers, applications",
2,398546,computer programmers,
3,177870,"computer occupations, all other",
4,83856,"software developers, systems software",
5,66873,management analysts,
6,53540,financial analysts,
7,52822,accountants and auditors,
8,44236,mechanical engineers,
9,40087,network and computer systems administrators,


In [14]:
reducedf.iloc[3]['Count'] #example of accessing a location

177870

In [15]:
%%timeit
for index, row in reducedf.iterrows():
    names = row['SOC_NAME'].split(",")
    if(names[0].endswith('*')):
        reducedf.set_value([index],['Name1'],(names[0][:-1]))
    if not (names[0].endswith('s')):
        reducedf.set_value([index],['Name1'],(names[0]+'s'))
    else:
        reducedf.set_value([index],['Name1'],names[0])

1 loop, best of 3: 919 ms per loop


In [16]:
reducedf

,Count,SOC_NAME,Name1
0,506523,computer systems analysts,computer systems analysts
1,414716,"software developers, applications",software developers
2,398546,computer programmers,computer programmers
3,177870,"computer occupations, all other",computer occupations
4,83856,"software developers, systems software",software developers
5,66873,management analysts,management analysts
6,53540,financial analysts,financial analysts
7,52822,accountants and auditors,accountants and auditors
8,44236,mechanical engineers,mechanical engineers
9,40087,network and computer systems administrators,network and computer systems administrators


In [17]:
cleandata1['SOC_NAME'].value_counts().count()

1585

In [18]:
(cleandata1.loc[(cleandata1['SOC_NAME']=='software developers, appllications')]) #an example of a query

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
894328,894329,CERTIFIED,"meridiansoft, inc.","software developers, appllications",software developer applications,Y,62421.0,2015.0,"LEWIS CENTER, OHIO",-83.010099,40.198388


This person messed up the SOC_NAME

In [19]:
reducedf['Name1'].value_counts()

software developers                                 37
securities                                          14
secondary school teachers                           14
electronics engineers                               13
sales representatives                               13
computer occupations                                11
computer software engineers                         10
elementary school teachers                          10
computer systems analysts                           10
mechanical engineers                                 9
special education teachers                           8
education administrators                             7
medical scientists                                   7
computer programmers                                 7
middle school teachers                               7
electrical engineers                                 7
civil engineers                                      6
cooks                                                6
foreign la

In [20]:
reducedf['Name1'].value_counts().count()

1134

We have now cut the number of names in half from the original number.

In [21]:
reducedf.sort_values(['Name1'])

,Count,SOC_NAME,Name1
1370,1,13-2011.01,13-2011.01s
1109,2,15-1121,15-1121s
1189,1,15-1132,15-1132s
1326,1,15-1199.01 sw quality assurance engnrs & testers,15-1199.01 sw quality assurance engnrs & testers
1429,1,15-1199.01,15-1199.01s
1337,1,"15-1199.08, business intelligence analysts",15-1199.08s
1558,1,17-2051,17-2051s
1472,1,17-2072,17-2072s
1340,1,27-3031,27-3031s
1509,1,29-1064.00-obstetricians and gynecologists,29-1064.00-obstetricians and gynecologists


In [22]:
reducedf['Name2'] = ""

In [23]:
%%timeit
regex = re.compile('[^a-z\s]')

for index, row in reducedf.iterrows():
    reducedf.set_value([index],['Name2'],(regex.sub('', row['Name1'])))

1 loop, best of 3: 918 ms per loop


In [24]:
reducedf.sort_values(['Name1'])

,Count,SOC_NAME,Name1,Name2
1370,1,13-2011.01,13-2011.01s,s
1109,2,15-1121,15-1121s,s
1189,1,15-1132,15-1132s,s
1326,1,15-1199.01 sw quality assurance engnrs & testers,15-1199.01 sw quality assurance engnrs & testers,sw quality assurance engnrs testers
1429,1,15-1199.01,15-1199.01s,s
1337,1,"15-1199.08, business intelligence analysts",15-1199.08s,s
1558,1,17-2051,17-2051s,s
1472,1,17-2072,17-2072s,s
1340,1,27-3031,27-3031s,s
1509,1,29-1064.00-obstetricians and gynecologists,29-1064.00-obstetricians and gynecologists,obstetricians and gynecologists


In [25]:
reducedf['Name2'].value_counts().count()

1118

Striping charactors did not help much.

At this point investigated a spellchecker in python. Was not able to get something to work within enviroment.

I am now going to consider removing entries that are unique, with a count of one.

In [26]:
dfName2Check = pd.DataFrame({'Name2': reducedf['Name2'].value_counts().index, 'Count':reducedf['Name2'].value_counts().values})

In [27]:
dfName2Check

,Count,Name2
0,37,software developers
1,14,securities
2,14,secondary school teachers
3,13,sales representatives
4,13,electronics engineers
5,11,computer occupations
6,10,computer software engineers
7,10,elementary school teachers
8,10,computer systems analysts
9,9,mechanical engineers


Theres a problem, I am not getting a real representation of the occurence of names in the data. I now need to do something where I can get the actual number of occurences.

In [28]:
cleandata1

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,1,CERTIFIED-WITHDRAWN,university of michigan,biochemists and biophysicists,postdoctoral research fellow,N,36067.00,2016.0,"ANN ARBOR, MICHIGAN",-83.743038,42.280826
1,2,CERTIFIED-WITHDRAWN,"goodman networks, inc.",chief executives,chief operating officer,Y,242674.00,2016.0,"PLANO, TEXAS",-96.698886,33.019843
2,3,CERTIFIED-WITHDRAWN,"ports america group, inc.",chief executives,chief process officer,Y,193066.00,2016.0,"JERSEY CITY, NEW JERSEY",-74.077642,40.728158
3,4,CERTIFIED-WITHDRAWN,"gates corporation, a wholly-owned subsidiary o...",chief executives,"regional presiden, americas",Y,220314.00,2016.0,"DENVER, COLORADO",-104.990251,39.739236
4,5,WITHDRAWN,peabody investments corp.,chief executives,president mongolia and india,Y,157518.40,2016.0,"ST. LOUIS, MISSOURI",-90.199404,38.627003
5,6,CERTIFIED-WITHDRAWN,burger king corporation,chief executives,"executive v p, global development and presiden...",Y,225000.00,2016.0,"MIAMI, FLORIDA",-80.191790,25.761680
6,7,CERTIFIED-WITHDRAWN,bt and mk energy and commodities,chief executives,chief operating officer,Y,91021.00,2016.0,"HOUSTON, TEXAS",-95.369803,29.760427
7,8,CERTIFIED-WITHDRAWN,"globo mobile technologies, inc.",chief executives,chief operations officer,Y,150000.00,2016.0,"SAN JOSE, CALIFORNIA",-121.886329,37.338208
8,9,CERTIFIED-WITHDRAWN,esi companies inc.,chief executives,president,Y,127546.00,2016.0,"MEMPHIS, TEXAS",NaN,NaN
9,10,WITHDRAWN,lessard international llc,chief executives,president,Y,154648.00,2016.0,"VIENNA, VIRGINIA",-77.265260,38.901222


Going to go back to this df and create a copy. Will then over write the Soc_Name with the reduced name I have.

In [29]:
cleandata2 = cleandata1.copy()

In [31]:
%%time
name = cleandata2.iloc[3002440]['SOC_NAME'] #example of accessing a location
print(name)
newname = reducedf.loc[(reducedf['SOC_NAME']==name)]
newname1 = newname.iloc[0]['Name2']
print(newname1)

software developers, applications
software developers
Wall time: 2.52 ms


Test code to test algorithm

At this point of development, I have switched to running the python notebook locally. I am having trouble with how slow the cloud service is, and I am not using spark yet.

In [38]:
%%timeit
errormap = defaultdict(list)

for index, row in cleandata2.iterrows():
    name = row['SOC_NAME']
    newname = reducedf.loc[(reducedf['SOC_NAME']==name)]
    try:
        newname1 = newname.iloc[0]['Name2']
        cleandata2.set_value([index],['SOC_NAME'],newname1)
    except:
        quicklist = ["Something went wrong", name, newname]
        errormap[index].append(quicklist)
    break

1 loop, best of 3: 1.03 s per loop


Here, we try to apply what we have reduced to the dataframe. We will see if we have made any progress, and what we can do next.

However, I have a very VERY large problem.
The logic to use the reduced names, takes 1 s.
With about 3000000 rows, that would be about 35 days.

I have no idea how to get around this right now. 

I think I might be able to do this, by converting the pandas dataframe into a spark rdd.
From there I could user map and map reduce functions.

But I would need a lot more time in order to do this. 

At this point. I have spent many hours on this.

I feel very defeated.

I am now going to see if there is some way I can streamline the algoritm.

In [48]:
%%timeit
for index, row in cleandata2.iterrows():
    try:
        cleandata2.set_value([index],['SOC_NAME'],reducedf.loc[(reducedf['SOC_NAME']==row['SOC_NAME'])].iloc[0]['Name2'])
    except:
        pass
    break

1 loop, best of 3: 997 ms per loop


Very little gains. I don't know what to do. We may have to switch over in entire to rapidminer.

In [37]:
cleandata2['SOC_NAME'].value_counts().count()
cleandata2['SOC_NAME'].value_counts()

computer systems analysts                                             506523
software developers, applications                                     414716
computer programmers                                                  398546
computer occupations, all other                                       177870
software developers, systems software                                  83856
management analysts                                                    66873
financial analysts                                                     53540
accountants and auditors                                               52822
mechanical engineers                                                   44236
network and computer systems administrators                            40087
database administrators                                                38982
market research analysts and marketing specialists                     37737
electronics engineers, except computer                                 36574

Debating what will happen if I remove all entries in the dataset with which a SOC_NAME that only occures once. How many entries will this remove? Can we ignore entries like this?
What about entries with an occurence count of 2? 3? Do these entries matter?
What do we do if they matter?

At this point, I will have to do as much as I can in python, and then switch over to rapidminer.

I have an idea on how I could go through the rows faster.

In [ ]:
%%timeit
num=3000000
i=1
while i < num:
    try:
        name = cleandata2.iloc[i]['SOC_NAME']
        newname = reducedf.loc[(reducedf['SOC_NAME']==name)].iloc[0]['Name2']
        #cleandata2.set_value(i,['SOC_NAME'], newname)
        i=i+1
    except:
        pass

its the setting of the values that takes the majority of the time.

<h1>Following code is for generating heat mape of lat and lon data

In [6]:
df_data_2

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,1,CERTIFIED-WITHDRAWN,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,N,36067.00,2016.0,"ANN ARBOR, MICHIGAN",-83.743038,42.280826
1,2,CERTIFIED-WITHDRAWN,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,242674.00,2016.0,"PLANO, TEXAS",-96.698886,33.019843
2,3,CERTIFIED-WITHDRAWN,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES,CHIEF PROCESS OFFICER,Y,193066.00,2016.0,"JERSEY CITY, NEW JERSEY",-74.077642,40.728158
3,4,CERTIFIED-WITHDRAWN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES,"REGIONAL PRESIDEN, AMERICAS",Y,220314.00,2016.0,"DENVER, COLORADO",-104.990251,39.739236
4,5,WITHDRAWN,PEABODY INVESTMENTS CORP.,CHIEF EXECUTIVES,PRESIDENT MONGOLIA AND INDIA,Y,157518.40,2016.0,"ST. LOUIS, MISSOURI",-90.199404,38.627003
5,6,CERTIFIED-WITHDRAWN,BURGER KING CORPORATION,CHIEF EXECUTIVES,"EXECUTIVE V P, GLOBAL DEVELOPMENT AND PRESIDEN...",Y,225000.00,2016.0,"MIAMI, FLORIDA",-80.191790,25.761680
6,7,CERTIFIED-WITHDRAWN,BT AND MK ENERGY AND COMMODITIES,CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,91021.00,2016.0,"HOUSTON, TEXAS",-95.369803,29.760427
7,8,CERTIFIED-WITHDRAWN,"GLOBO MOBILE TECHNOLOGIES, INC.",CHIEF EXECUTIVES,CHIEF OPERATIONS OFFICER,Y,150000.00,2016.0,"SAN JOSE, CALIFORNIA",-121.886329,37.338208
8,9,CERTIFIED-WITHDRAWN,ESI COMPANIES INC.,CHIEF EXECUTIVES,PRESIDENT,Y,127546.00,2016.0,"MEMPHIS, TEXAS",NaN,NaN
9,10,WITHDRAWN,LESSARD INTERNATIONAL LLC,CHIEF EXECUTIVES,PRESIDENT,Y,154648.00,2016.0,"VIENNA, VIRGINIA",-77.265260,38.901222


In [3]:
lon = []
lon = df_data_2['lon'].values,df_data_2['lat'].values
lon

(array([-83.7430378, -96.6988856, -74.0776417, ..., -74.90989  ,
        -76.7802528, -84.3879824]),
 array([ 42.2808256,  33.0198431,  40.7281575, ...,  40.6367682,
         39.4195499,  33.7489954]))

In [14]:
#for long, lat in df_data_2['lon'].values for in df_data_2['lat'].values
geo = [list(a) if a!=['nan', 'nan'] for a in zip(df_data_2['lon'].values,df_data_2['lat'].values)]

SyntaxError: invalid syntax (<ipython-input-14-3d1e6af45427>, line 2)

In [13]:
geo

[[-83.743037799999996, 42.2808256],
 [-96.698885599999997, 33.019843100000003],
 [-74.077641700000001, 40.728157500000002],
 [-104.990251, 39.739235799999996],
 [-90.199404200000004, 38.627002500000003],
 [-80.1917902, 25.7616798],
 [-95.369802799999988, 29.7604267],
 [-121.88632859999998, 37.338208200000004],
 [nan, nan],
 [-77.265260400000003, 38.901222499999996],
 [-79.995886400000003, 40.440624799999995],
 [-84.247211599999986, 43.615582500000002],
 [nan, nan],
 [-80.1917902, 25.7616798],
 [-104.95081409999999, 39.617210100000001],
 [-77.429129799999998, 39.006699299999994],
 [-88.231481299999999, 43.011678400000001],
 [-118.24368490000002, 34.052234200000001],
 [-121.95523559999999, 37.354107899999995],
 [-77.046921400000002, 38.804835499999996],
 [-118.1597929, 34.0005691],
 [-118.24368490000002, 34.052234200000001],
 [-117.1610838, 32.715738000000002],
 [-117.08419550000001, 32.6400541],
 [-80.1917902, 25.7616798],
 [-121.95523559999999, 37.354107899999995],
 [-87.62979820000001

In [ ]:
numpy.random.seed(0)
seaborn.set()
uniform_data = lon
ax = seaborn.heatmap(uniform_data)


KeyboardInterrupt



In [10]:
uniform_data

array([[ 0.5488135 ,  0.71518937,  0.60276338,  0.54488318,  0.4236548 ,
         0.64589411,  0.43758721,  0.891773  ,  0.96366276,  0.38344152,
         0.79172504,  0.52889492],
       [ 0.56804456,  0.92559664,  0.07103606,  0.0871293 ,  0.0202184 ,
         0.83261985,  0.77815675,  0.87001215,  0.97861834,  0.79915856,
         0.46147936,  0.78052918],
       [ 0.11827443,  0.63992102,  0.14335329,  0.94466892,  0.52184832,
         0.41466194,  0.26455561,  0.77423369,  0.45615033,  0.56843395,
         0.0187898 ,  0.6176355 ],
       [ 0.61209572,  0.616934  ,  0.94374808,  0.6818203 ,  0.3595079 ,
         0.43703195,  0.6976312 ,  0.06022547,  0.66676672,  0.67063787,
         0.21038256,  0.1289263 ],
       [ 0.31542835,  0.36371077,  0.57019677,  0.43860151,  0.98837384,
         0.10204481,  0.20887676,  0.16130952,  0.65310833,  0.2532916 ,
         0.46631077,  0.24442559],
       [ 0.15896958,  0.11037514,  0.65632959,  0.13818295,  0.19658236,
         0.36872517,  